In [1]:
!pip install transformers==4.35.0 accelerate==0.24.0 peft==0.6.0 bitsandbytes==0.41.0 trl==0.7.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of datasets to determine which version is compatible w

In [8]:
from datasets import load_dataset
from transformers import TrainingArguments, BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig
from trl import SFTTrainer

In [3]:
train_params = TrainingArguments(
 optim = "paged_adamw_32bit",
 learning_rate = 3e-4,
 weight_decay = 0.01,
 lr_scheduler_type = 'cosine',
 warmup_ratio = 0.03,
 gradient_accumulation_steps = 4,
 bf16 = True,
 gradient_checkpointing = True,
 label_smoothing_factor = 0.1,
 neftune_noise_alpha = 5,
 per_device_train_batch_size = 8,
 per_device_eval_batch_size = 8,
 max_grad_norm=2,
 group_by_length=True,
 num_train_epochs=3,
 output_dir = '/model_outputs',
 save_steps = 50,
 logging_steps = 10
 )


In [4]:
quantize_params = BitsAndBytesConfig (
 use_4bit = True,
 bnb_4bit_compute_dtype = 'float16',
 bnb_4bit_quant_type = 'nf4',
 use_nested_quant = False
 )


In [5]:
lora_params = LoraConfig (
 r = 64,
 lora_alpha = 8,
 lora_dropout = 0.1
 )


In [7]:
model = Auto.from_pretrained(
 pretrained_model_name_or_path = 'Deci/DeciLM-7B',
 quantization_config=quantize_params,
 device_map='auto'
 )
tokenizer = LlamaTokenizer.from_pretrained('meta-llama/Llama-2-7b')


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ImportError: ignored

In [9]:
tokenizer = AutoTokenizer.from_pretrained('Deci/DeciLM-7B')


In [10]:
tune_data = load_dataset("Cesar7980/fingpt_chatglm2_sentiment_instruction_lora_ft_dataset")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/76772 [00:00<?, ? examples/s]

In [ ]:
sft = SFTTrainer (
 model = model,
 args = train_params,
 train_dataset = tune_data,
 tokenizer = tokenizer
 peft_config = lora_params,
 max_seq_length = 128,
 dataset_text_field = 'text',
 packing = True
 )


In [ ]:
sft.train()
sft.model.save_pretrained('/path/to/tuned_model.csv')

In [ ]:
Use Reference Notebook to fill in the details: https://colab.research.google.com/drive/1PEQyJO1-f6j0S_XJ8DV50NkpzasXkrzd?usp=sharing